In [199]:
#import libraries

import pandas as pd
import numpy as np
import pandas as pd
import re
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

# Load file

In [200]:
#import file about the french museums infos

Paris_exhibitions=pd.read_csv('Cleaned_data/Paris_exhibitions.csv', sep=',', encoding='UTF8')

pd.set_option('display.max_columns', 20)

# Pre-processing

In [201]:
#gets user input

genre = input("Hi! What is the genre of the exhibition you're interested in?")
audience = input("Are you interested in a Public enfant / Public adolescent & adulte / Tout public / 'Public enfant & adolescent / Public adulte / Public adolescent exhibition?")

#uses user input to print out information
print("You're interested in", genre, "and by an exhibition for", audience)

You're interested in Art contemporain and by an exhibition for Tout public


In [202]:
exhibition_suggestion = Paris_exhibitions.loc[(Paris_exhibitions["genre"] == genre) & (Paris_exhibitions["audience"] == audience)]
exhibition_suggested = exhibition_suggestion.sample()
exhibition_suggested

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
418,35973,https://www.paris.fr/evenements/alphonse-mucha...,Alphonse Mucha au Grand Palais Immersif,"Une exposition numérique d'Alphonse Mucha, « É...","<p>Figure majeure de l'Art nouveau, l'artiste ...",2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,exposition",Grand Palais Immersif,"110, rue de Lyon",75012,Paris,"48.8512849988655, 2.37022301292524",gratuit,Tout public,Art contemporain,0


In [203]:
#retrieve the index number of the exhibition suggested

index_suggested = exhibition_suggested.index.item()

#retrieve the exhibition information

exhibition_suggestion.loc[index_suggested:index_suggested]

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
418,35973,https://www.paris.fr/evenements/alphonse-mucha...,Alphonse Mucha au Grand Palais Immersif,"Une exposition numérique d'Alphonse Mucha, « É...","<p>Figure majeure de l'Art nouveau, l'artiste ...",2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,exposition",Grand Palais Immersif,"110, rue de Lyon",75012,Paris,"48.8512849988655, 2.37022301292524",gratuit,Tout public,Art contemporain,0


In [204]:
#print out information about the exhibition suggested
print("Here is one exhibition you could like:\n", str(exhibition_suggested['titre']), "\nDescription:", str(exhibition_suggested['chapeau']), "\nPlace:", str(exhibition_suggested['nom_du_lieu']))

Here is one exhibition you could like:
 418    Alphonse Mucha au Grand Palais Immersif
Name: titre, dtype: object 
Description: 418    Une exposition numérique d'Alphonse Mucha, « É...
Name: chapeau, dtype: object 
Place: 418    Grand Palais Immersif
Name: nom_du_lieu, dtype: object


# Machine learning

In [205]:
def find_similar_exhibitions(index_suggested):
    df = Paris_exhibitions.drop_duplicates(subset=["chapeau"]).reset_index(drop=True)
    strings = list(df["chapeau"])
    strings_without_digits = []

    # Uses a regex in order to remove the numbers
    pattern = r"\d+"

    for s in strings:
        if isinstance(s, str):
            s_without_digits = re.sub(pattern, "", s)
            strings_without_digits.append(s_without_digits)
        else:
            strings_without_digits.append(s)

    # Convert the keywords into a list
    keywords = list(df["mots_clés"])

    # Create CountVectorizer and fit_transform the texts
    vectorizer = CountVectorizer()
    texts = ["" if pd.isna(text) else text for text in strings_without_digits]
    vectors = vectorizer.fit_transform(texts)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    headers = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=texts)

    # Create CountVectorizer and fit_transform the keywords
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(keywords)
    vocabulary = vectorizer.get_feature_names_out()

    # Create DataFrame with the vectors
    keywords = pd.DataFrame.sparse.from_spmatrix(vectors, columns=vocabulary, index=keywords)

    # Apply PCA on headers
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(headers.to_numpy())

    # Create DataFrame with PCA components for headers
    headers_pca = pd.DataFrame(vectors_pca, index=df.index)

    # Apply PCA on keywords
    pca = PCA(n_components=15)  # Specify the number of components to keep
    vectors_pca = pca.fit_transform(keywords.to_numpy())

    # Create DataFrame with PCA components for keywords
    keywords_pca = pd.DataFrame(vectors_pca, index=keywords.index)

    # Concatenate the PCA components
    concatenated_df = pd.concat([headers_pca, keywords_pca])

    # Fit Nearest Neighbors model
    neigh = NearestNeighbors(n_neighbors=6)
    neigh.fit(concatenated_df)

    # Find similar exhibitions for a specific index (35 in this case)
    distances, indices = neigh.kneighbors(concatenated_df.iloc[index_suggested:index_suggested+1])

    # Return the similar exhibitions
    return Paris_exhibitions.iloc[indices[0]]

In [206]:
find_similar_exhibitions(index_suggested)

,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
418,35973,https://www.paris.fr/evenements/alphonse-mucha...,Alphonse Mucha au Grand Palais Immersif,"Une exposition numérique d'Alphonse Mucha, « É...","<p>Figure majeure de l'Art nouveau, l'artiste ...",2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,exposition",Grand Palais Immersif,"110, rue de Lyon",75012,Paris,"48.8512849988655, 2.37022301292524",gratuit,Tout public,Art contemporain,0
29,36235,https://www.paris.fr/evenements/fabrique-ateli...,[fabrique] Atelier création d’accessoires festifs,On se prépare à la fête ! En couleurs et en mu...,<p>Pas de fête ou carnaval sans accessoire !</...,2023-06-03 19:00:00+02:00,2023-06-21 21:30:00+02:00,"Atelier,Enfants,Loisirs",La Maison des métallos,"94, rue Jean-Pierre Timbaud",75011,Paris,"48.867741, 2.377855",payant,Tout public,Atelier,18
807,35759,https://www.paris.fr/evenements/expo-exploser-...,Expo - Exploser le cadre,Pourquoi « exploser le cadre » ? Comment oser ...,<p><strong>Vernissage le mercredi 10 mai à 18h...,2023-05-09 02:00:00+02:00,2023-05-27 01:59:59+02:00,"Art contemporain,Expo",Centre Paris Anim' Espace Beaujon,"208, rue du Faubourg Saint-Honoré",75008,Paris,"48.875214, 2.304807",gratuit,Tout public,Art contemporain,17
852,36116,https://www.paris.fr/evenements/ateliers-en-an...,Ateliers en anglais à la bibliothèque Andrée C...,Venez vous exprimer et échanger en anglais. Ch...,<p>Ateliers en anglais de niveau débutant à in...,2023-05-17 20:30:00+02:00,2023-06-14 21:30:00+02:00,Atelier,Bibliothèque Andrée Chedid,"36, rue Emeriau",75015,Paris,"48.8500160206342, 2.28643097089903",gratuit,Public adulte,Atelier,28
1535,36519,https://www.paris.fr/evenements/dancing-with-m...,Dancing With Myself / Supersonic 80's Party,"Concerts gratuits, Nuits rock et Disquaire à P...",<p>Dancing With Myself… Ta soirée 80s préférée...,2023-06-04 02:00:00+02:00,2023-06-04 09:00:00+02:00,Clubbing,SUPERSONIC,9 Rue Biscornet,75012,Paris,"48.8502469839762, 2.36970702296574",payant,Tout public,Clubbing,0
1708,26898,https://www.paris.fr/evenements/concert-gospel...,Concert Gospel Hélios à l’Église de la Madeleine,Ne manquez pas cette incroyable performance du...,<h2>Concert Gospel Hélios à l’Église de la Mad...,2022-12-31 23:45:00+01:00,2023-10-01 01:00:00+02:00,"Concert,Musique",Église de La Madeleine,place de la Madeleine,75008,Paris,"48.869876, 2.32354",payant,Public adolescent & adulte,Concert,273


In [207]:
new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)


# Simulation

In [208]:
def recommend_exhibition(exhibition_suggested):
    index = 0  # Counter to keep track of the suggested exhibitions
    while True:
        if index < len(exhibition_suggested):
            new_exhibition_suggestion = exhibition_suggested.iloc[index]
            while True:
                response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                if response.lower() == "y":
                    similar_exhibitions = find_similar_exhibitions(index_suggested)
                    return pd.DataFrame(similar_exhibitions)
                elif response.lower() == "n":
                    new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                    new_exhibition_suggested = new_exhibition_suggestion.sample()
                    print("Here is another exhibition you could like:\n")
                    print("Title:", new_exhibition_suggested['titre'].values[0])
                    print("Description:", new_exhibition_suggested['chapeau'].values[0])
                    print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    response = input("Is the suggested exhibition relevant for you? Yes/No (Y/N): ")
                    if response.lower() == "y":
                        similar_exhibitions = find_similar_exhibitions(index_suggested)
                        return pd.DataFrame(similar_exhibitions)
                    elif response.lower() == "n":
                        new_exhibition_suggestion = exhibition_suggestion.drop(index=index_suggested)
                        new_exhibition_suggested = new_exhibition_suggestion.sample()
                        print("Here is another exhibition you could like:\n")
                        print("Title:", new_exhibition_suggested['titre'].values[0])
                        print("Description:", new_exhibition_suggested['chapeau'].values[0])
                        print("Location:", new_exhibition_suggested['nom_du_lieu'].values[0])
                    break
                else:
                    print("Invalid response. Please enter 'Y' for Yes or 'N' for No.")
        else:
            print("No more exhibitions to suggest.")
            return


# Call the function and pass the exhibition DataFrame
recommend_exhibition(exhibition_suggested)


Here is another exhibition you could like:

Title: Agora « Comment faire face aux questions de biodiversité et de permaculture en milieux urbains ? » avec le collectif l'Aberiette et Thierry Boutonnier
Description: À Malakoff, le centre d’art investit 4 000 m2 d’espace public, au pied de la cité Stalingrad et de la supérette, lieu d’expérimentation du centre d’art, avec une programmation propice au partage et à l’interaction artistique et sociale.
Location: Quartier Stalingrad - la supérette centre d'art contemporain de Malakoff
Here is another exhibition you could like:

Title: Le sens de la fête, Installation de Marianne Villière "Dreamhouse et Ritournelles"
Description: Dreamhouse et Ritournelles / Installation / Phrases et mots dessinés dans les rues de Nanterre à l'aide de pochoirs ainsi que de peinture écologique et drapeaux fixés dans les fourreaux des entrées de la ville de Nanterre.
Location: Parc des Anciennes-Mairies


,id,url,titre,chapeau,description,date_de_début,date_de_fin,mots_clés,nom_du_lieu,adresse_du_lieu,code_postal,ville,coordonnées_géographiques,type_de_prix,audience,genre,durée
418,35973,https://www.paris.fr/evenements/alphonse-mucha...,Alphonse Mucha au Grand Palais Immersif,"Une exposition numérique d'Alphonse Mucha, « É...","<p>Figure majeure de l'Art nouveau, l'artiste ...",2023-06-03 22:00:00+02:00,2023-06-04 02:00:00+02:00,"Art contemporain,exposition",Grand Palais Immersif,"110, rue de Lyon",75012,Paris,"48.8512849988655, 2.37022301292524",gratuit,Tout public,Art contemporain,0
29,36235,https://www.paris.fr/evenements/fabrique-ateli...,[fabrique] Atelier création d’accessoires festifs,On se prépare à la fête ! En couleurs et en mu...,<p>Pas de fête ou carnaval sans accessoire !</...,2023-06-03 19:00:00+02:00,2023-06-21 21:30:00+02:00,"Atelier,Enfants,Loisirs",La Maison des métallos,"94, rue Jean-Pierre Timbaud",75011,Paris,"48.867741, 2.377855",payant,Tout public,Atelier,18
807,35759,https://www.paris.fr/evenements/expo-exploser-...,Expo - Exploser le cadre,Pourquoi « exploser le cadre » ? Comment oser ...,<p><strong>Vernissage le mercredi 10 mai à 18h...,2023-05-09 02:00:00+02:00,2023-05-27 01:59:59+02:00,"Art contemporain,Expo",Centre Paris Anim' Espace Beaujon,"208, rue du Faubourg Saint-Honoré",75008,Paris,"48.875214, 2.304807",gratuit,Tout public,Art contemporain,17
852,36116,https://www.paris.fr/evenements/ateliers-en-an...,Ateliers en anglais à la bibliothèque Andrée C...,Venez vous exprimer et échanger en anglais. Ch...,<p>Ateliers en anglais de niveau débutant à in...,2023-05-17 20:30:00+02:00,2023-06-14 21:30:00+02:00,Atelier,Bibliothèque Andrée Chedid,"36, rue Emeriau",75015,Paris,"48.8500160206342, 2.28643097089903",gratuit,Public adulte,Atelier,28
1535,36519,https://www.paris.fr/evenements/dancing-with-m...,Dancing With Myself / Supersonic 80's Party,"Concerts gratuits, Nuits rock et Disquaire à P...",<p>Dancing With Myself… Ta soirée 80s préférée...,2023-06-04 02:00:00+02:00,2023-06-04 09:00:00+02:00,Clubbing,SUPERSONIC,9 Rue Biscornet,75012,Paris,"48.8502469839762, 2.36970702296574",payant,Tout public,Clubbing,0
1708,26898,https://www.paris.fr/evenements/concert-gospel...,Concert Gospel Hélios à l’Église de la Madeleine,Ne manquez pas cette incroyable performance du...,<h2>Concert Gospel Hélios à l’Église de la Mad...,2022-12-31 23:45:00+01:00,2023-10-01 01:00:00+02:00,"Concert,Musique",Église de La Madeleine,place de la Madeleine,75008,Paris,"48.869876, 2.32354",payant,Public adolescent & adulte,Concert,273
